# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118387e+02     1.690921e+00
 * time: 0.41190505027770996
     1     1.157809e+01     8.566835e-01
 * time: 2.4014999866485596
     2    -1.212195e+01     9.897637e-01
 * time: 3.155385971069336
     3    -3.461552e+01     7.355064e-01
 * time: 4.259565114974976
     4    -4.846742e+01     5.447242e-01
 * time: 5.362986087799072
     5    -5.724080e+01     1.861844e-01
 * time: 6.458285093307495
     6    -5.993453e+01     7.813146e-02
 * time: 7.170528173446655
     7    -6.087450e+01     7.362736e-02
 * time: 7.873545169830322
     8    -6.131396e+01     5.878579e-02
 * time: 8.574692010879517
     9    -6.163072e+01     3.719785e-02
 * time: 9.281658172607422
    10    -6.185066e+01     2.873163e-02
 * time: 9.994160175323486
    11    -6.197161e+01     2.019939e-02
 * time: 10.758285999298096
    12    -6.204377e+01     1.765688e-02
 * time: 11.51046109199524
    13    -6.209419e+01     1.762655e-02
 * time: 12.219422101974487
  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671070
    AtomicLocal         -18.8557723
    AtomicNonlocal      14.8522685
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485376 
    Xc                  -19.3336821

    total               -62.261666456923
